# Custom Classification weight loading with PytorchWildlife

This tutorial guides you on how to use PyTorchWildlife for custom weight loading. We will go through the process of setting up the environment, defining the detection model, defining the custom classification model, as well as performing inference and saving the results in different ways.

## Prerequisites
Install PytorchWildlife running the following commands:
```bash
conda create -n pytorch_wildlife python=3.8 -y
conda activate pytorch_wildlife
pip install PytorchWildlife
pip install wget
```
Also, make sure you have a CUDA-capable GPU if you intend to run the model on a GPU. This notebook can also run on CPU.

## Importing libraries
First, we'll start by importing the necessary libraries and modules.

In [ ]:
import os
import numpy as np
import os
from PIL import Image
import torch
from torch.utils.data import DataLoader
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife.models import classification as pw_classification
from PytorchWildlife.data import transforms as pw_trans
from PytorchWildlife.data import datasets as pw_data 
from PytorchWildlife import utils as pw_utils
import wget

## Model Initialization
We will initialize the MegaDetectorV6 model for image detection. This model is designed for detecting animals in images.

In [ ]:
# Setting the device to use for computations ('cuda' indicates GPU)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda":
    torch.cuda.set_device(0)

detection_model = pw_detection.MegaDetectorV6(device=DEVICE, pretrained=True, version="yolov9c")

# Uncomment the following line to use MegaDetectorV5 instead of MegaDetectorV6
#detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True, version="a")

In [ ]:
# Defining the classification model
example_weights = "https://zenodo.org/records/13357337/files/AI4GAmazonClassification_v0.0.0.ckpt?download=1"
filename = wget.download(example_weights)
class_names = {
        0: 'Dasyprocta',
        1: 'Bos',
        2: 'Pecari',
        3: 'Mazama',
        4: 'Cuniculus',
        5: 'Leptotila',
        6: 'Human',
        7: 'Aramides',
        8: 'Tinamus',
        9: 'Eira',
        10: 'Crax',
        11: 'Procyon',
        12: 'Capra',
        13: 'Dasypus',
        14: 'Sciurus',
        15: 'Crypturellus',
        16: 'Tamandua',
        17: 'Proechimys',
        18: 'Leopardus',
        19: 'Equus',
        20: 'Columbina',
        21: 'Nyctidromus',
        22: 'Ortalis',
        23: 'Emballonura',
        24: 'Odontophorus',
        25: 'Geotrygon',
        26: 'Metachirus',
        27: 'Catharus',
        28: 'Cerdocyon',
        29: 'Momotus',
        30: 'Tapirus',
        31: 'Canis',
        32: 'Furnarius',
        33: 'Didelphis',
        34: 'Sylvilagus',
        35: 'Unknown'
    }

classification_model = pw_classification.CustomWeights(weights=filename, class_names=class_names, device=DEVICE)

In [ ]:
tgt_img_path = os.path.join(".","demo_data","imgs","10050028_0.JPG")
img = np.array(Image.open(tgt_img_path).convert("RGB"))
results = detection_model.single_image_detection(img, tgt_img_path)
pw_utils.save_detection_images(results, os.path.join(".","demo_output"), overwrite=False)

## Batch Image Detection
Next, we'll demonstrate how to process multiple images in batches. This is useful when you have a large number of images and want to process them efficiently.

In [ ]:
tgt_folder_path = os.path.join(".","demo_data","imgs")
det_results = detection_model.batch_image_detection(tgt_folder_path, batch_size=16)

In [ ]:
clf_dataset = pw_data.DetectionCrops(
            det_results,
            transform=pw_trans.Classification_Inference_Transform(target_size=224),
            path_head=""
        )
clf_loader = DataLoader(clf_dataset, batch_size=32, shuffle=False, 
                                pin_memory=True, num_workers=0, drop_last=False)
clf_results = classification_model.batch_image_classification(clf_loader, id_strip=tgt_folder_path)


In [ ]:
pw_utils.save_detection_classification_json(det_results=det_results,
                                                        clf_results=clf_results,
                                                        det_categories=detection_model.CLASS_NAMES,
                                                        clf_categories=classification_model.CLASS_NAMES,
                                                        output_path="results.json")